In [46]:
import os
import time
import requests
import io
from PIL import Image
import hashlib

In [24]:
from selenium import webdriver
DRIVER_PATH = '/Users/guillaume/Desktop/Scraping/chromedriver'
wd = webdriver.Chrome(executable_path=DRIVER_PATH)

In [35]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [43]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image =Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [47]:
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [54]:
# search_and_download("Alma Mater Columbia",driver_path=DRIVER_PATH,number_images=150)
# search_and_download("Butler Library Columbia",driver_path=DRIVER_PATH,number_images=100)
search_and_download("Lion statue Columbia",driver_path=DRIVER_PATH,number_images=100)

Found: 100 search results. Extracting links from 0:100
Found: 100 image links, done!
SUCCESS - saved https://commencement.columbia.edu/sites/default/files/styles/hero/public/basic-page-campus-maps-hero-1240x720_0.jpg?itok=LSvM4OuJ - as ./images/lion_statue_columbia/9156340a11.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSnMArxM2DdwBQzR3jLqDXzxPvsergh4LNYOg&usqp=CAU - as ./images/lion_statue_columbia/a2208ecb7a.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQfcZ-iXw9UgZAKNYq_MZEGVvf6F_3GxkW66g&usqp=CAU - as ./images/lion_statue_columbia/757dc51258.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSITa3XaXHPh_-LabtJh7V-aO7P1Jw18oprow&usqp=CAU - as ./images/lion_statue_columbia/2a8d2e0451.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcT309ZM0ioSIxk6GYBkfUVqmD8jURC_-1ueJg&usqp=CAU - as ./images/lion_statue_columbia/afe893afa7.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.

SUCCESS - saved https://blogs.cul.columbia.edu/outdoorsculpture/files/2017/10/Lion1.jpg - as ./images/lion_statue_columbia/0216e29b15.jpg
SUCCESS - saved https://columbia.giftplans.org/files/2314/9434/3061/daf.jpg - as ./images/lion_statue_columbia/14f81d049d.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRnan6HBM45WX0P1sHoFyVg1GSfLgKCM3hBiA&usqp=CAU - as ./images/lion_statue_columbia/a60550a66a.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQWilOYP7OMRoDk1Fo_SxjJL7X-aMgf8s0DwQ&usqp=CAU - as ./images/lion_statue_columbia/304ca02894.jpg
SUCCESS - saved https://gocolumbialions.com/images/2017/3/8/KTKZAIKCGZMIXIE.20170308200038.jpg - as ./images/lion_statue_columbia/d77545a5e1.jpg
SUCCESS - saved https://lh3.googleusercontent.com/proxy/d2ckGkiBwswlGiMiiw08vqjLlQmH_4hep-KpNgHAbLsc_lsPDh6PyjFX9cQV4k5IrR539XFfS6u5F06Wk0zxfkxIKgYvxkB1q7C_WGHayEShJubRseKB2g - as ./images/lion_statue_columbia/c3bc9610b2.jpg
SUCCESS - saved https://encryp

SUCCESS - saved https://img.geocaching.com/waymarking/display/41333126-a34c-4edb-8034-0750eb42a5a0.jpg - as ./images/lion_statue_columbia/89ba6f0d08.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-5rWq9XDRx1qk6x4cO3F4XRm5RpJP7AlOTg&usqp=CAU - as ./images/lion_statue_columbia/ac787468ed.jpg
ERROR - Could not save https://stadiumjourney.com/wp-content/uploads/2015/11/IMG_0698-400x284.jpg - cannot identify image file <_io.BytesIO object at 0x7fe2d5048e30>
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRzgqAJpzooWA19F8KmQZ22RhS1xGtUzImd1A&usqp=CAU - as ./images/lion_statue_columbia/8a35a24548.jpg
